In [ ]:
# 1 : wczytanie, preprocessing i drzewo ID3

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.metrics import accuracy_score, classification_report

# 1) Wczytujemy dane (możesz pominąć, jeśli df jest już w RAM)
df = pd.read_csv('warehouse_priority_dataset.csv')

# 2) Podział X / y
X = df.drop(columns='priority')
y = df['priority']

categorical = ['size', 'type']
numeric     = [c for c in X.columns if c not in categorical]

# 3) Pipeline: One-Hot + drzewo (criterion='entropy' ≡ ID3)
preprocess = ColumnTransformer([
    ('cat', OneHotEncoder(sparse=False), categorical),
    ('num', 'passthrough', numeric)
])

model = Pipeline([
    ('prep', preprocess),
    ('tree', DecisionTreeClassifier(criterion='entropy',
                                    random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.25, random_state=42
)

model.fit(X_train, y_train)

# 4) Ewaluacja
pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, pred):.2f}\n")
print(classification_report(y_test, pred))

# 5) Reguły drzewa 
feature_names = model.named_steps['prep'].get_feature_names_out()
print("\n⤵  Drzewo decyzyjne (reguły):\n")
print(export_text(model.named_steps['tree'],
                 feature_names=list(feature_names)))

# 6) Wizualizacje 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

print("\n\n" + "="*50)
print("WIZUALIZACJA MODELU DLA PREZENTACJI")
print("="*50)

# Wykres drzewa decyzyjnego
plt.figure(figsize=(20, 12))
plot_tree(model.named_steps['tree'], 
          feature_names=list(feature_names), 
          class_names=list(model.named_steps['tree'].classes_),
          filled=True, 
          rounded=True, 
          precision=2)
plt.title('Drzewo Decyzyjne', fontsize=24)
plt.tight_layout()
plt.savefig('drzewo_decyzyjne.png', bbox_inches='tight', dpi=300)
print("\nWykres drzewa zapisany jako 'drzewo_decyzyjne.png'")

# Confusion Matrix
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_test, pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.named_steps['tree'].classes_)
disp.plot(cmap=plt.cm.Blues, values_format='d')
plt.title('Confusion Matrix', fontsize=20)
plt.tight_layout()
plt.savefig('macierz_konfuzji.png', bbox_inches='tight', dpi=300)
print("Macierz konfuzji zapisana jako 'macierz_konfuzji.png'")

# Ważność cech
plt.figure(figsize=(12, 8))
importances = model.named_steps['tree'].feature_importances_
indices = np.argsort(importances)[::-1]
plt.bar(range(len(importances)), importances[indices])
plt.xticks(range(len(importances)), 
           [feature_names[i] for i in indices], 
           rotation=90)
plt.xlabel('Cechy')
plt.ylabel('Ważność')
plt.title('Ważność Cech', fontsize=20)
plt.tight_layout()
plt.savefig('waznosc_cech.png', bbox_inches='tight', dpi=300)
print("Wykres ważności cech zapisany jako 'waznosc_cech.png'")

# 7) Podsumowanie
print("\n\n" + "="*50)
print("PODSUMOWANIE WYNIKÓW")
print("="*50)

# Statystyki
accuracy = accuracy_score(y_test, pred)
report = classification_report(y_test, pred, output_dict=True)

# Wyniki klas
class_results = {}
for cls in model.named_steps['tree'].classes_:
    class_results[cls] = {
        'precision': report[cls]['precision'],
        'recall': report[cls]['recall'],
        'f1-score': report[cls]['f1-score']
    }

# Podsumowanie tekstowe
print(f"\n Dokładność modelu: {accuracy:.4f} ({accuracy*100:.1f}%)")
print(f" Liczba instancji treningowych: {len(X_train)}")
print(f" Liczba instancji testowych: {len(X_test)}")
print(f" Głębokość drzewa decyzyjnego: {model.named_steps['tree'].get_depth()}")
print(f" Liczba węzłów drzewa: {model.named_steps['tree'].tree_.node_count}")

print("\n Wyniki dla poszczególnych klas:")
for cls, metrics in class_results.items():
    print(f"  • {cls}:")
    print(f"    - Precision: {metrics['precision']:.4f}")
    print(f"    - Recall: {metrics['recall']:.4f}")
    print(f"    - F1-score: {metrics['f1-score']:.4f}")

print("\n Najważniejsze cechy:")
for i in indices[:5]:  # Top 5 cech
    print(f"  • {feature_names[i]}: {importances[i]:.4f}")

# Wyświetlenie wszystkich wykresów na koniec
plt.show()